In [ ]:
os.makedirs('/walmart/drive', exist_ok=True)

from google.colab import drive
drive.mount('/walmart/drive')


In [ ]:
import pandas as pd
import os
from sklearn import preprocessing
import time
seconds = time.time()
%matplotlib inline

number = "0123456789"

# CSV files names
csv_files = [
    "Monday-WorkingHours.pcap_ISCX",
    "Tuesday-WorkingHours.pcap_ISCX",
    "Wednesday-workingHours.pcap_ISCX",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX",
    "Friday-WorkingHours-Morning.pcap_ISCX",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX",
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX",
]

# Headers of columns
main_labels = ["Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets",
   "Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min",
   "Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std",
   "Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max",
   "Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags",
   "Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std",
   "Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count",
   "ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","faulty-Fwd Header Length","Fwd Avg Bytes/Bulk",
   "Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes",
   "Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd",
   "min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label","External IP"]

main_labels2 = main_labels.copy()
header_str = ",".join(main_labels) + "\n"
flag = True

base_path = "/walmart/drive/My Drive/walmart/CSVs/"
output_path = "/walmart/drive/My Drive/walmart/all_data.csv"


# Remove existing merged file if present
if os.path.exists(output_path):
    os.remove(output_path)

for i in range(len(csv_files)):
    tmp_file_path = os.path.join("/content", f"{i}.csv")

    # Write header to temp file
    with open(tmp_file_path, "w") as ths:
        ths.write(header_str)

        input_file = os.path.join(base_path, csv_files[i] + ".csv")
        with open(input_file, "r", encoding='utf-8', errors='ignore') as file:
            while True:
                line = file.readline()
                if not line:
                    break
                try:
                    if line[0] in number:
                        line = line.replace(" – ", " - ")
                        line = line.replace("inf", "0")
                        line = line.replace("Infinity", "0")
                        line = line.replace("NaN", "0")
                        ths.write(line)
                except:
                    continue

    # Read cleaned file
    df = pd.read_csv(tmp_file_path, low_memory=False)
    df = df.fillna(0)

    string_features = ["Flow Bytes/s", "Flow Packets/s"]
    for col in string_features:
        df[col] = df[col].replace('Infinity', -1)
        df[col] = df[col].replace('NaN', 0)
        numeric_values = []
        for val in df[col]:
            try:
                numeric_values.append(int(float(val)))
            except:
                numeric_values.append(val)
        df[col] = numeric_values

    # Detect string (categorical) columns
    str_feats = []
    for col in main_labels2:
        if df[col].dtype == "object":
            str_feats.append(col)

    try:
        str_feats.remove("Label")
    except:
        print("Warning: 'Label' column not found in string_features list")

    labelencoder_X = preprocessing.LabelEncoder()
    for col in str_feats:
        try:
            df[col] = labelencoder_X.fit_transform(df[col])
        except:
            df[col] = df[col].replace("Infinity", -1)

    # Drop faulty column
    df = df.drop(main_labels2[61], axis=1)

    # Merge to all_data.csv
    if flag:
        df.to_csv(output_path, index=False)
        flag = False
    else:
        df.to_csv(output_path, index=False, header=False, mode="a")

    # Remove temp file
    os.remove(tmp_file_path)
    print(f"The pre-processing phase of the {csv_files[i]} file is completed.\n")

print("Mission accomplished!")
print("Total operation time: = ", time.time() - seconds, "seconds")


This process may take 5 to 10 minutes, depending on the performance of your computer.



The pre-processing phase of the  Monday-WorkingHours.pcap_ISCX  file is completed.

The pre-processing phase of the  Tuesday-WorkingHours.pcap_ISCX  file is completed.

The pre-processing phase of the  Wednesday-workingHours.pcap_ISCX  file is completed.

The pre-processing phase of the  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX  file is completed.

The pre-processing phase of the  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX  file is completed.

The pre-processing phase of the  Friday-WorkingHours-Morning.pcap_ISCX  file is completed.

The pre-processing phase of the  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX  file is completed.

The pre-processing phase of the  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX  file is completed.

mission accomplished!
Total operation time: =  434.0625455379486 seconds
